In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame as df 
import math
import random as rand


from sklearn.metrics import silhouette_score,calinski_harabasz_score,davies_bouldin_score
import copy 
from tabulate import tabulate
import random

In [2]:
df_new = pd.read_json(open("../Dataset/train.json", "r"))
df_new.head()
df_new["num_photos"] = df_new["photos"].apply(len)
df_new["num_features"] = df_new["features"].apply(len)
df_new["num_description_words"] = df_new["description"].apply(lambda x: len(x.split(" ")))
df_new["created"] = pd.to_datetime(df_new["created"])
df_new["created_year"] = df_new["created"].dt.year
df_new["created_month"] = df_new["created"].dt.month
df_new["created_day"] = df_new["created"].dt.day

In [3]:
# data=pd.read_csv("total_data_na.csv")
# data=pd.read_csv("../input/Iris.csv")
# data=data.dropna()
# data=data.drop(['Species','Id'],axis=1)
# data=data.drop["JURISDICTIONNAME"]
# data= data.drop("PLAYER",axis=1)
num_feats = ["bathrooms", "bedrooms", "latitude", "longitude", "price",
             "num_photos", "num_features", "num_description_words",
             "created_year", "created_month", "created_day"]
data= df_new[num_feats]
y = df_new["interest_level"]
data = data.reset_index()
data.drop('index', inplace=True, axis = 1)
data.head()

,bathrooms,bedrooms,latitude,longitude,price,num_photos,num_features,num_description_words,created_year,created_month,created_day
0,1.0,1,40.7108,-73.9539,2400,12,7,77,2016,6,16
1,1.0,2,40.7513,-73.9722,3800,6,6,131,2016,6,1
2,1.0,2,40.7575,-73.9625,3495,6,6,119,2016,6,14
3,1.5,3,40.7145,-73.9425,3000,5,0,95,2016,6,24
4,1.0,0,40.7439,-73.9743,2795,4,4,41,2016,6,28


In [4]:
# data=pd.read_csv("total_data_na.csv")
# data=pd.read_csv("../Dataset/school_district_breakdowns_1.csv")
# data=data.dropna()
# data=data.drop(['Species'],axis=1)
# d_names=data["PLAYER"]
# data=data.drop("jurisdiction_name",axis=1)
# d_names=data["PLAYER"]
# data=data.drop("JURISDICTION NAME",axis=1)
# data=data.drop["JURISDICTIONNAME"]
# data= data.drop("PLAYER",axis=1)
data

,bathrooms,bedrooms,latitude,longitude,price,num_photos,num_features,num_description_words,created_year,created_month,created_day
0,1.0,1,40.7108,-73.9539,2400,12,7,77,2016,6,16
1,1.0,2,40.7513,-73.9722,3800,6,6,131,2016,6,1
2,1.0,2,40.7575,-73.9625,3495,6,6,119,2016,6,14
3,1.5,3,40.7145,-73.9425,3000,5,0,95,2016,6,24
4,1.0,0,40.7439,-73.9743,2795,4,4,41,2016,6,28
...,...,...,...,...,...,...,...,...,...,...,...
49347,1.0,3,40.8433,-73.9396,2800,5,3,131,2016,4,5
49348,1.0,2,40.8198,-73.9578,2395,5,7,70,2016,4,2
49349,1.0,1,40.5765,-73.9554,1850,3,11,81,2016,4,26
49350,1.0,2,40.7448,-74.0017,4195,5,5,118,2016,4,19


In [5]:
encoder=['00','01','11','10']

In [9]:
data = data[:1000]

In [10]:
data

,bathrooms,bedrooms,latitude,longitude,price,num_photos,num_features,num_description_words,created_year,created_month,created_day
0,1.0,1,40.7108,-73.9539,2400,12,7,77,2016,6,16
1,1.0,2,40.7513,-73.9722,3800,6,6,131,2016,6,1
2,1.0,2,40.7575,-73.9625,3495,6,6,119,2016,6,14
3,1.5,3,40.7145,-73.9425,3000,5,0,95,2016,6,24
4,1.0,0,40.7439,-73.9743,2795,4,4,41,2016,6,28
...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,1,40.7075,-74.0059,3175,0,4,9,2016,6,12
996,2.0,2,40.7943,-73.9675,1995,8,1,168,2016,6,25
997,1.0,1,40.7515,-73.9413,2490,8,5,102,2016,6,26
998,1.0,3,40.6759,-73.9226,2200,0,0,9,2016,6,12


In [11]:
#Frequency Based Encoding
details= dict()
for j in data.columns:
  col=data[j]
  sortx = dict()
  a=list()
  for i in col:
    if i not in sortx:
      sortx[i] = 1
      a.append(i)
    else:
      sortx[i]+=1
  a.sort()
  p=0
  s=0
  i=0
  l=0
  while p<len(data):
    m=int(sortx[a[l]])
    if  s+m <=(math.ceil(len(data)/len(encoder))) :
      s+=m
      sortx[a[l]]=encoder[i]
    else:
      if l == 0:
        sortx[a[l]]=encoder[i]
        i+=1
      else:
        i+=1
        sortx[a[l]]=encoder[i]
      s=0
    l+=1
    p+=m
  details[j]=sortx



In [12]:
#Replacing the data in table with encoding
data1 = df(columns = data.columns)
for i in range(len(data)):
  t=[]
  p=data.loc[i]
  for j in range(len(p)):
    s=details[data.columns[j]]
    t.append(s[p[j]])
  ar = pd.Series(t, index = data1.columns)
  data1 = data1.append(ar,ignore_index = True)

data1

/var/folders/j7/0xc74mdd0vn8k3qvwmg5vl840000gn/T/ipykernel_74861/1138767711.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data1 = data1.append(ar,ignore_index = True)


,bathrooms,bedrooms,latitude,longitude,price,num_photos,num_features,num_description_words,created_year,created_month,created_day
0,01,01,00,10,00,10,10,01,00,01,11
1,01,11,01,11,11,11,11,10,00,01,00
2,01,11,11,11,11,11,11,10,00,01,01
3,01,11,00,10,01,01,00,11,00,01,10
4,01,00,01,11,01,01,11,00,00,01,10
...,...,...,...,...,...,...,...,...,...,...,...
995,00,01,00,00,11,00,11,00,00,01,01
996,01,11,10,11,00,11,00,10,00,01,10
997,01,01,11,10,01,11,11,11,00,01,10
998,01,11,00,10,00,00,00,00,00,01,01


In [13]:
#Generating final encoded string
enc=[]
for i in range(len(data1)):
  p=data1.loc[i]
  t=""
  for j in p:
    t+=j
  enc.append(t)
enc

['0101001000101001000111',
 '0111011111111110000100',
 '0111111111111110000101',
 '0111001001010011000110',
 '0100011101011100000110',
 '0111010110011110000110',
 '0111010110111101000101',
 '0100101000011101000100',
 '0001010100010001000101',
 '0111000010100110000110',
 '0100111001011101000100',
 '0111110011011011000100',
 '0100110000011011000110',
 '0101000001110011000101',
 '0111111110101000000101',
 '0100001111100001000111',
 '0111111111110110000101',
 '0111010110111010000111',
 '0101100111010010000111',
 '0101111001000000000110',
 '0100011100011100000110',
 '0111010010111101000101',
 '0101110110111010000101',
 '0101010111111110000100',
 '0101111001010111000101',
 '0111010110111101000111',
 '0111000010011101000100',
 '0111111011110110000110',
 '0100111100011111000110',
 '0111110111011011000100',
 '0111000011111111000110',
 '0100110100000100000111',
 '0111101110000001000101',
 '0101000101010110000101',
 '0111000000110001000100',
 '0111001101100100000101',
 '0101110001011101000110',
 

In [14]:
len(enc[0])

22

In [15]:
r_ind=[6, 12, 18]

In [16]:
#Vertical Split into 6
t=[]
for i in range(len(enc)):
  s=0
  for j in r_ind:
    t.append(enc[i][s:j])
    s=j
  t.append(enc[i][s:])
  enc[i]=t
  t=[]
for i in enc:
  print(i)


['010100', '100010', '100100', '0111']
['011101', '111111', '111000', '0100']
['011111', '111111', '111000', '0101']
['011100', '100101', '001100', '0110']
['010001', '110101', '110000', '0110']
['011101', '011001', '111000', '0110']
['011101', '011011', '110100', '0101']
['010010', '100001', '110100', '0100']
['000101', '010001', '000100', '0101']
['011100', '001010', '011000', '0110']
['010011', '100101', '110100', '0100']
['011111', '001101', '101100', '0100']
['010011', '000001', '101100', '0110']
['010100', '000111', '001100', '0101']
['011111', '111010', '100000', '0101']
['010000', '111110', '000100', '0111']
['011111', '111111', '011000', '0101']
['011101', '011011', '101000', '0111']
['010110', '011101', '001000', '0111']
['010111', '100100', '000000', '0110']
['010001', '110001', '110000', '0110']
['011101', '001011', '110100', '0101']
['010111', '011011', '101000', '0101']
['010101', '011111', '111000', '0100']
['010111', '100101', '011100', '0101']
['011101', '011011', '110

In [17]:
#Function to add null boundary and returns string
def nullbound(n,winsize,t1):
  if n%2!=0:
          if winsize%2!=0:
            t1 = "0"*(winsize//2) + t1 + "0"*(winsize//2)
          else:
            t1 = "0"*((winsize//2)+1) + t1 + "0"*(winsize//2)
  else:
          if winsize%2!=0:
            t1 = "0"*(winsize//2) + t1 + "0"*(winsize//2)
          else:
            t1 = "0"*((winsize//2)+1) + t1 + "0"*(winsize//2)
  return t1

In [18]:
#generating binary rule
def rg(rule):
  brule=bin(rule).replace("0b","").zfill(2**winsize)
  brule=brule[::-1]
  return brule 

In [19]:
my_list = [
252645135, 
252645195,
252645336,
252645360,
252652815,
252656655,
252656880,
252691215,
252691440,
252695055,
252698895,
252702735,
252702960,
253678110,
254611245,
254618925,
255594255,
255652080,
256577295,
256577355,
259190799,
259526415,
260509455,
260960271,
263458575,
263458635,
264441615,
264499440,
265482450,
267390735,
267390795,
267390960,
267416079,
267422991,
267448335,
267448359,
267448560,
505290270,
505336350,
509222490,
517136850,
517140690,
521018910,
568251090,
570174960,
570217200,
695036265,
755961615,
755969295,
756011535,
756019215,
756994590,
757927725,
757931565,
757935405,
757939245,
764265357,
764269197,
764273037,
764276877,
921712188,
1010580540,
1016918172,
1016918172,
1511938590,
1231776105,
1259293455,
1259293515,
1259293560,
1259293680,
1263225615,
1263225675,
1267157835,
1267157880,
1271089995,
1752787305,
1768515945,
1799965515,
1831415085,
1921479288,
1924428408,
2018211960,
2018212020,
2018212080,
2019194970,
2021161080,
2072267535,
2072267595,
2076199695,
2076199755,
2076199920,
2218767375,
2218767540,
2218767600,
2222699700,
2222699760,
2273806215,
2275772325,
2276755215,
2276755275,
2276755335,
252648975,
2370538887,
2463552210,
2526451350,
2777130375,
2783028705,
3027809415,
3031741620,
3035673615,
3035673780,
3063191190,
3530698098,
3537028050,
3537035730,
3537972705,
3538955760,
3539005680,
3724750095,
3726716205,
3777826605,
3785744805,
3789677025,
4027518936,
4027544304,
4027576335,
4027576560,
4030467855,
4031508660,
4034007024,
4035440880,
4038389940,
4039315215,
4040348370,
4041289185,
4042264560,
4042272240,
263458575,
264441615,
265482450,
267390795,
267416079,
267448335,
267448560,
505336350,
517136850,
521018910,
570174960,
695036265,
755969295,
756019215,
757927725,
757935405,
764265357,
764273037,
921712188,
1016918172,
1511938590,
1259293455,
1259293560,
1263225615,
1267157835]
# for i in range(len(rule_list)):
#   rule_list[i]=str(rule_list[i])

rule_list=random.sample(my_list,2)
print(rule_list)



[4035440880, 1267157835]


In [20]:
#For finding the length of the next encoding(as in 2-bit, 3-bit encoding)
def findn(d):
    i=1
    while 2**i<d:
      i+=1
    return i

def cy_enc(l,index):
    return   bin(index).replace("0b","").zfill(l)

In [21]:
from sklearn.cluster import KMeans
from sklearn import metrics                                                                                                                  

In [22]:
#Will apply the CA rule and generate all possible cycles and returns nested list
def rule_app1(a1,winsize,brule):
  final_array = []
  a=list(set(a1))
  n=len(a[0])
  current_array=[]
  while(a):
    t1=a[0]
    flag=0
    while(not flag):
      if current_array == []:
        current_array.append(t1)
        a.remove(t1)
        
      else:
        t1=nullbound(n,winsize,t1)
        t2=""

        for j in range(n):
          check=int(str(t1[j:j+winsize]),2)
          t2+=brule[check]

        t1=t2

        if t1 not in current_array:
            if t1 in a:
              a.remove(t1)
              current_array.append(t1)
              
        else:
            flag=1
        

    final_array.append(current_array)
    current_array=[]
    cycle=1
    # print(cycle)
    cycle+=1
  return final_array

In [23]:

#Clustering, checking scores 252702960
winsize = 5
split=4

# rule_list=[267390795,267416079]
# rule_list=[256577355,259190799]
# rule_list=[252691440,2218767375]
# rule_list=[[1511938590,2218767375],[2276755335,1259293455],[2276755215,255652080]]
#rule_list=[[1511938590,2218767375]]

tmc=[]
tma=[]
enc1 = copy.deepcopy(enc)
for p in range(int(len(rule_list)/2)):
  rule = rg(int(rule_list[0]))
  fc = {}

  tr = []
  for i in range(split):
    for j in range(len(enc1)):
      tr.append(enc1[j][i])
    fc[i]=rule_app1(tr,winsize,rule)
    tr = []

  for i in range(split):
    t=fc[i]
    s=findn(len(t))
    for j in range(len(enc1)):
      plt=enc1[j][i]
      l=0
      while(plt not in t[l]):
        l+=1
      enc1[j][i]= cy_enc(s,l)
  rule=rg(int(rule_list[1]))
  for i in range(len(enc1)):
    t=""
    for j in enc1[i]:
      t+=j
    enc1[i]=t
  # print(len(enc1))
  fa=rule_app1(enc1,winsize,rule)
  #print(fa)
  kl=copy.deepcopy(enc1)
  for i in range(len(enc1)):
    j=0
    while j <len(fa):
      if enc1[i] in fa[j]:
        enc1[i]=j
        break
      else:
        j+=1

  
  X=data.to_numpy()
  #print(len(enc1),len(X))
  print(p)
  
  if (len(set(enc1))>1):
      pl=silhouette_score(X,enc1,metric="euclidean")
      print(rule_list)
      # print(rule_list[2*p+1])
      print("silhoutte:",pl)
      # print("calinski:",calinski_harabasz_score(X,enc1))
      # print("davies:",davies_bouldin_score(X,enc1))

      clust=KMeans(n_clusters=len(set(enc1)),random_state=42)
      clust.fit(X)
      labels=clust.labels_
      
      pl1=metrics.silhouette_score(X, labels, metric='euclidean')
      # print("Kmeans silhoutte :" ,pl1)
      # print(len(set(labels)))
      # print("Kmeans calinski :", (metrics.calinski_harabasz_score(X, labels)))
      # print("Kmeans davies_bouldin_score: ",metrics.davies_bouldin_score(X, labels))
      print("-----------")
      tmc.append([pl,pl1])
      if tma==[] or tma[0][0]<pl:
        tma=[[pl,pl1]]
  #enc1=copy.deepcopy(enc)




  



0
[4035440880, 1267157835]
silhoutte: -0.6470088254602704


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


-----------


In [24]:

head=["record for SC for CA algo","Corresponding SC for Kmeans"]
print(tabulate(tmc,headers=head,tablefmt="grid"))
head=["max SC for CA algo","Corresponding SC for Kmeans"]
print(tabulate(tma,headers=head,tablefmt="grid"))

+-----------------------------+-------------------------------+
|   record for SC for CA algo |   Corresponding SC for Kmeans |
+=============================+===============================+
|                   -0.647009 |                      0.518217 |
+-----------------------------+-------------------------------+
+----------------------+-------------------------------+
|   max SC for CA algo |   Corresponding SC for Kmeans |
+======================+===============================+
|            -0.647009 |                      0.518217 |
+----------------------+-------------------------------+


In [25]:
kls=copy.deepcopy(kl)
t1=copy.deepcopy(fa) 
for i in range(len(fa)-2):
  #print(len(t1),"###")
  kl=copy.deepcopy(kls)  
  fa1=copy.deepcopy(t1)
  idx=0
  for j in range(len(fa1)):
    if len(fa1[j])<len(fa1[idx]):
      idx=j
      
  t=copy.deepcopy(t1) 
  removed=t.pop(idx)
  #print(removed)
  ml=[]

  for j in removed:
    ml.append([kls.index(j),j])
  ml=sorted(ml)
  for j in ml:
    kl.remove(j[1])
  #print(ml)
  for j in range(len(fa1[idx])):
    p=[]
    a,b=ml[0]
    kl.insert(a,b)
    X=data.copy(deep=True)
    if len(ml)>1:
        for h in ml[1:]:
          X.drop(h[0],inplace=True)
      #print(len(X))
    X=X.to_numpy()
    for k in range(len(t)):
      m=copy.deepcopy(t)
      m[k].append(b)

      kle=copy.deepcopy(kl)
      for z in range(len(kle)):
        c=0
        while c <len(m):
          if kle[z] in m[c]:
            kle[z]=c
            break
          else:
            c+=1
      pl=silhouette_score(X,kle,metric="euclidean")
      p.append(pl)
    #print(p.index(max(p)))
    #print(t[p.index(max(p))])
    t[p.index(max(p))].append(fa1[idx][j])
    ml.pop()
    t1=t
    l=[]
    for i in t1:
      l.append(len(i))
    #print(l)

In [26]:
len(kl)

1000

In [27]:
for i in range(len(kls)):
    j=0
    while j <len(t1):
      if kls[i] in t1[j]:
        kls[i]=j
        break
      else:
        j+=1
X=data.to_numpy()
pl=silhouette_score(X,kls,metric="euclidean")
print(rule_list)
print("silhoutte:",pl)
print("calinski:",calinski_harabasz_score(X,kls))
print("davies:",davies_bouldin_score(X,kls))
clust=KMeans(n_clusters=len(set(kls)),random_state=42)
clust.fit(X)
labels=clust.labels_
      
pl1=metrics.silhouette_score(X, labels, metric='euclidean')
# print("Kmeans silhoutte :" ,pl1)
# print("Kmeans calinski :", (metrics.calinski_harabasz_score(X, labels)))
# print("Kmeans davies_bouldin_score: ",metrics.davies_bouldin_score(X, labels))
# print(len(set(labels)))


[4035440880, 1267157835]
silhoutte: 0.0021449898059869603
calinski: 0.10025807233318823
davies: 48.523734233609034


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [28]:
my_data=[[len(fa),tma[0][0],tma[0][1],pl,pl1]]
head=["Initial no.of clusters", "Silhoutte score for our algo","silhoutte score for kmeans"," new silhoutte score","new silhoute for kmeans"]
print(tabulate(my_data, headers=head, tablefmt="grid"))

+--------------------------+--------------------------------+------------------------------+------------------------+---------------------------+
|   Initial no.of clusters |   Silhoutte score for our algo |   silhoutte score for kmeans |    new silhoutte score |   new silhoute for kmeans |
+==========================+================================+==============================+========================+===========================+
|                       10 |                      -0.647009 |                     0.518217 |             0.00214499 |                  0.967895 |
+--------------------------+--------------------------------+------------------------------+------------------------+---------------------------+


In [143]:
p=[]
for i in fa:
  p.append(len(i))
q=[]
for i in t1:
  q.append(len(i))
print("before :",p)
print("after: ",q)

print(t1)

before : [3, 1, 2, 1, 1]
after:  [5, 3]
[['011', '101', '001', '110', '010'], ['100', '111', '000']]


In [144]:
from sklearn.cluster import KMeans, MeanShift, DBSCAN, Birch
from sklearn import datasets
X=data.to_numpy()


In [145]:
clust_model = MeanShift(bandwidth=50)
clust_model.fit(X)
labels = clust_model.labels_
pl1=metrics.silhouette_score(X, labels, metric='euclidean')
print("Meanshift silhoutte :" ,pl1)
print("Meanshift calinski :", (metrics.calinski_harabasz_score(X, labels)))
print("Meanshift davies_bouldin_score: ",metrics.davies_bouldin_score(X, labels))


ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)

In [27]:
clust_model = DBSCAN(min_samples=2, eps=100)
clust_model.fit(X)
# Evaluating model's performance

labels = clust_model.labels_
# pl1=metrics.silhouette_score(X, labels, metric='euclidean')
print("DBSCAN silhoutte :" ,pl1)
# print("DBSCAN calinski :", (metrics.calinski_harabasz_score(X, labels)))
# print("DBSCAN davies_bouldin_score: ",metrics.davies_bouldin_score(X, labels))


DBSCAN silhoutte : 0.46821835837440073


In [28]:
clust_model = Birch(n_clusters=2,branching_factor=1000,threshold=1.5)
clust_model.fit(X)
labels = clust_model.labels_
pl1=metrics.silhouette_score(X, labels, metric='euclidean')
print("Birch silhoutte :" ,pl1)
print("Birch calinski :", (metrics.calinski_harabasz_score(X, labels)))
print("Birch davies_bouldin_score: ",metrics.davies_bouldin_score(X, labels))


Birch silhoutte : 0.7663611080988648
Birch calinski : 94.42893185124181
Birch davies_bouldin_score:  0.43429835583227294


In [29]:
X


array([[11, 4, 0.36, ..., False, 11, 100],
       [124, 70, 0.56, ..., False, 124, 100],
       [38, 26, 0.68, ..., False, 38, 100],
       ...,
       [6, 4, 0.67, ..., False, 6, 100],
       [0, 0, 0.0, ..., False, 0, 0],
       [0, 0, 0.0, ..., False, 0, 0]], dtype=object)

In [30]:
#Clustering, checking scores 252702960
winsize = 5
split=5

#rule_list=[267390795,267416079]
#rule_list=[256577355,259190799]
#rule_list=[252691440,2218767375]
# rule_list=[[1511938590,2218767375],[2276755335,1259293455],[2276755215,255652080]]
#rule_list=[[1511938590,2218767375]]

tmc=[]
tma=[]
enc1 = copy.deepcopy(enc)
for p in range(int(len(rule_list)/2)):
  rule = rg(int(rule_list[0]))
  fc = {}

  tr = []
  for i in range(split):
    for j in range(len(enc1)):
      tr.append(enc1[j][i])
    fc[i]=rule_app1(tr,winsize,rule)
    tr = []

  for i in range(split):
    t=fc[i]
    s=findn(len(t))
    for j in range(len(enc1)):
      plt=enc1[j][i]
      l=0
      while(plt not in t[l]):
        l+=1
      enc1[j][i]= cy_enc(s,l)
  rule=rg(int(rule_list[1]))
  for i in range(len(enc1)):
    t=""
    for j in enc1[i]:
      t+=j
    enc1[i]=t
  # print(len(enc1))
  fa=rule_app1(enc1,winsize,rule)
  #print(fa)
  kl=copy.deepcopy(enc1)
  for i in range(len(enc1)):
    j=0
    while j <len(fa):
      if enc1[i] in fa[j]:
        enc1[i]=j
        break
      else:
        j+=1

  
  X=data.to_numpy()
  #print(len(enc1),len(X))
  print(p)
  
  if (len(set(enc1))>1):
      pl=silhouette_score(X,enc1,metric="euclidean")
      # print(rule_list[2*p])
      # print(rule_list[2*p+1])
      # print("silhoutte:",pl)
      # print("calinski:",calinski_harabasz_score(X,enc1))
      # print("davies:",davies_bouldin_score(X,enc1))

      clust=KMeans(n_clusters=len(set(enc1)),random_state=42)
      clust.fit(X)
      labels=clust.labels_
      
      pl1=metrics.silhouette_score(X, labels, metric='euclidean')
      # print("Kmeans silhoutte :" ,pl1)
      print(len(set(labels)))
      # print("Kmeans calinski :", (metrics.calinski_harabasz_score(X, labels)))
      # print("Kmeans davies_bouldin_score: ",metrics.davies_bouldin_score(X, labels))
      print("-----------")
      tmc.append([pl,pl1])
      if tma==[] or tma[0][0]<pl:
        tma=[[pl,pl1]]
  #enc1=copy.deepcopy(enc)




  



0
6
-----------


In [31]:
kls=copy.deepcopy(kl)
t1=copy.deepcopy(fa) 
for i in range(len(fa)-2):
  #print(len(t1),"###")
  kl=copy.deepcopy(kls)  
  fa1=copy.deepcopy(t1)
  idx=0
  for j in range(len(fa1)):
    if len(fa1[j])<len(fa1[idx]):
      idx=j
      
  t=copy.deepcopy(t1) 
  removed=t.pop(idx)
  #print(removed)
  ml=[]

  for j in removed:
    ml.append([kls.index(j),j])
  ml=sorted(ml)
  for j in ml:
    kl.remove(j[1])
  #print(ml)
  for j in range(len(fa1[idx])):
    p=[]
    a,b=ml[0]
    kl.insert(a,b)
    X=data.copy(deep=True)
    if len(ml)>1:
        for h in ml[1:]:
          X.drop(h[0],inplace=True)
      #print(len(X))
    X=X.to_numpy()
    for k in range(len(t)):
      m=copy.deepcopy(t)
      m[k].append(b)

      kle=copy.deepcopy(kl)
      for z in range(len(kle)):
        c=0
        while c <len(m):
          if kle[z] in m[c]:
            kle[z]=c
            break
          else:
            c+=1
      pl=silhouette_score(X,kle,metric="euclidean")
      p.append(pl)
    #print(p.index(max(p)))
    #print(t[p.index(max(p))])
    t[p.index(max(p))].append(fa1[idx][j])
    ml.pop()
    t1=t
    l=[]
    for i in t1:
      l.append(len(i))
    #print(l)

In [32]:
len(kl)

32

In [33]:
for i in range(len(kls)):
    j=0
    while j <len(t1):
      if kls[i] in t1[j]:
        kls[i]=j
        break
      else:
        j+=1
X=data.to_numpy()
pl=silhouette_score(X,kls,metric="euclidean")
print("silhoutte:",pl)
print("calinski:",calinski_harabasz_score(X,kls))
print("davies:",davies_bouldin_score(X,kls))
clust=KMeans(n_clusters=len(set(kls)),random_state=42)
clust.fit(X)
labels=clust.labels_
      
pl1=metrics.silhouette_score(X, labels, metric='euclidean')
print("Kmeans silhoutte :" ,pl1)
print("Kmeans calinski :", (metrics.calinski_harabasz_score(X, labels)))
print("Kmeans davies_bouldin_score: ",metrics.davies_bouldin_score(X, labels))
print(len(set(labels)))

silhoutte: 0.443505982241705
calinski: 4.60603342090343
davies: 1.3405901839679435
Kmeans silhoutte : 0.7663611080988648
Kmeans calinski : 94.42893185124181
Kmeans davies_bouldin_score:  0.43429835583227294
2


In [34]:
my_data=[[len(fa),tma[0][0],tma[0][1],pl,pl1]]
head=["Initial no.of clusters", "Silhoutte score for our algo","silhoutte score for kmeans"," new silhoutte score","new silhoute for kmeans"]
print(tabulate(my_data, headers=head, tablefmt="grid"))

+--------------------------+--------------------------------+------------------------------+------------------------+---------------------------+
|   Initial no.of clusters |   Silhoutte score for our algo |   silhoutte score for kmeans |    new silhoutte score |   new silhoute for kmeans |
+==========================+================================+==============================+========================+===========================+
|                        6 |                      -0.302546 |                     0.635624 |               0.443506 |                  0.766361 |
+--------------------------+--------------------------------+------------------------------+------------------------+---------------------------+


In [ ]:
p=[]
for i in fa:
  p.append(len(i))
q=[]
for i in t1:
  q.append(len(i))
print("before :",p)
print("after: ",q)

print(t1)

before : [25, 1, 1]
after:  [25, 2]
[['0100010100001000010', '0010001010000101010', '0000000000010000011', '1000000000000100010', '0001000010100001101', '0000000000010000001', '0001001001000001101', '0001100101100001000', '0100100000111000010', '0100010000000100010', '0100000001110110010', '0101101000000011001', '0010000000000010010', '0001000100000001101', '0100001100000100010', '0111011110010111110', '0011011000000011001', '0101010101110000001', '0000011100000000001', '0001001000010001000', '0110011000100000001', '0010000001000110010', '0101100000000011001', '0000100000000000001', '0010001000001000010'], ['0001000000000001101', '0001000001010001100']]


In [ ]:
from sklearn.cluster import KMeans, MeanShift, DBSCAN, Birch
from sklearn import datasets
X=data.to_numpy()


In [ ]:
clust_model = MeanShift(bandwidth=50)
clust_model.fit(X)
labels = clust_model.labels_
pl1=metrics.silhouette_score(X, labels, metric='euclidean')
print("Meanshift silhoutte :" ,pl1)
print("Meanshift calinski :", (metrics.calinski_harabasz_score(X, labels)))
print("Meanshift davies_bouldin_score: ",metrics.davies_bouldin_score(X, labels))


Meanshift silhoutte : 0.46821835837440073
Meanshift calinski : 350.2929624179578
Meanshift davies_bouldin_score:  0.23522706626769807


In [ ]:
clust_model = DBSCAN(min_samples=2, eps=100)
clust_model.fit(X)
# Evaluating model's performance

labels = clust_model.labels_
# pl1=metrics.silhouette_score(X, labels, metric='euclidean')
print("DBSCAN silhoutte :" ,pl1)
# print("DBSCAN calinski :", (metrics.calinski_harabasz_score(X, labels)))
# print("DBSCAN davies_bouldin_score: ",metrics.davies_bouldin_score(X, labels))


DBSCAN silhoutte : 0.46821835837440073


In [ ]:
clust_model = Birch(n_clusters=2,branching_factor=1000,threshold=1.5)
clust_model.fit(X)
labels = clust_model.labels_
pl1=metrics.silhouette_score(X, labels, metric='euclidean')
print("Birch silhoutte :" ,pl1)
print("Birch calinski :", (metrics.calinski_harabasz_score(X, labels)))
print("Birch davies_bouldin_score: ",metrics.davies_bouldin_score(X, labels))


Birch silhoutte : 0.7663611080988648
Birch calinski : 94.42893185124181
Birch davies_bouldin_score:  0.43429835583227294


In [ ]:
X


array([[ 11.  ,   4.  ,   0.36, ...,   0.  ,  11.  , 100.  ],
       [124.  ,  70.  ,   0.56, ...,   0.  , 124.  , 100.  ],
       [ 38.  ,  26.  ,   0.68, ...,   0.  ,  38.  , 100.  ],
       ...,
       [  6.  ,   4.  ,   0.67, ...,   0.  ,   6.  , 100.  ],
       [  0.  ,   0.  ,   0.  , ...,   0.  ,   0.  ,   0.  ],
       [  0.  ,   0.  ,   0.  , ...,   0.  ,   0.  ,   0.  ]])